In [19]:
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
#nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import matplotlib.pyplot as plt
import imaplib,email
ps= PorterStemmer()
#nltk.download('punkt')
from nltk.tokenize import word_tokenize as wt
dt = pd.read_csv('spam.csv',encoding='latin-1')
stop_words= stopwords.words('english')
X= dt['v2']
#print(X)
listN=[]
listFinal=[]
for each in X:
    #print(each)
    rep = (re.sub('[^a-zA-Z]',' ',each)).split()
    rep1 = [ps.stem(elem) for elem in rep if elem not in set(stop_words) if len(elem) > 3 ]
    listN.append(rep1)
listFinal = [' '.join(each) for each in listN]
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=6000)
features = cv.fit_transform(listFinal).shape[1]
X= cv.fit_transform(listFinal).toarray()
Y= dt['v1'].values
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score as ac,confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
knn= KNeighborsClassifier(n_neighbors=15,metric= 'minkowski',p=2)
dsc =DecisionTreeClassifier(criterion='entropy',random_state=1,max_leaf_nodes=36)
gb= GaussianNB()
rC= RandomForestClassifier(n_estimators=100,criterion='entropy', random_state=1,max_leaf_nodes=123)
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.15,random_state=1)
rC.fit(X_train,y_train)
#print(ac(y_test,rC.predict(X_test)))
s= imaplib.IMAP4_SSL('imap.gmail.com')
s.login('XXXXXXXXXXXXXXXXx@gmail.com','XXXXXXXXXXX')# GIVE YOUR OWN GAMILID and PASSWORD
# If the connection is not established open Gmail -->go to googleaccount-->Sign-in And Security-->Allow less secure apps:--> Convert to ON
#print(s.list())
#s.select('"[Gmail]/Spam"')
#s.select('"LOR Sent by Jyotirmoy Gupta"')
s.select('"Inbox"')
typ1,data = s.search(None,'ALL')
mail_id=data[0]
#print(mail_id)
idList = mail_id.split()
#print(len(idList))
first_email_id = int(idList[0])
latest_email_id = int(idList[-1])
#print(first_email_id)
m=1
data=''
listFinal=[]
j=0
for i in range(latest_email_id,first_email_id,-1):
    #typ,data = s.fetch(b'50937','(RFC822)')
    #print(s.fetch(idList[i-1],'(RFC822)'))
    data= (s.fetch(idList[i-1],'(RFC822)'))
    j+=1
    dec=(data[1][0][1])
    k = email.message_from_bytes(dec)
   # print(k['from'],k['Subject'],k['date'])
    #if  k['from'].find('c.boviard@')>0:
    listFinal.append(k['Subject'])
        #print(len(listFinal))
    if len(listFinal)>20:
        break  
finalList=[]
col=0
for each in listFinal:
    rep = re.sub('[^A-Za-z]',' ',each).split()
    repStemmed = [ps.stem(elem) for elem in rep if elem not in set(stop_words) if len(elem) > 2]
    #print(repStemmed)
    strJoined = ' '.join(repStemmed)
    finalList.append(strJoined)
cv1 = CountVectorizer(max_features=120)
#print(cv1.fit_transform(finalList).toarray())
#print(cv1.fit_transform(finalList).shape)
if cv1.fit_transform(finalList).shape[1] < cv.fit_transform(listFinal).shape[1]:
    col =  abs(features - cv1.fit_transform(finalList).shape[1])
    #print(features,cv1.fit_transform(finalList).shape[1],col)
    row= cv1.fit_transform(finalList).shape[0]
    #print(row)
    val=row*col
    #print(val)
A= np.zeros(val).reshape(row,col)
#print(A)
#print((cv1.fit_transform(finalList)))
#A= np.zeros(112266).reshape(21,5346)
#A= np.zeros(112266).reshape(21,5346)
XFinalList= np.hstack((cv1.fit_transform(finalList).toarray(),A))
#print(XFinalList.shape) 
print(rC.predict(XFinalList))

['ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham'
 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham']
